In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
import glob
import json
import os
from tqdm.notebook import tqdm
import csv

import pandas as pd

### 한글 깨짐 해결

In [3]:
IN_PATH = '/content/drive/MyDrive/CUAI_winter/frame_data/'
OUT_PATH = '/content/drive/MyDrive/CUAI_winter/frame_csv/'

In [4]:
if not os.path.exists(OUT_PATH):
    os.makedirs(OUT_PATH)

In [5]:
json_list = sorted(glob.glob(IN_PATH+'*.json'))

for json_name in tqdm(json_list): # json 순회
    with open(json_name, "r") as f:
        try:
            json_file = json.load(f, encoding="utf-8")
        except:
            print("Response content is not valid JSON")
            continue

    csv_name = json_name.split('/')[-1].replace('json', 'csv')
    

    frame_list = list(json_file.keys())
    
    csv_data = []

    for frame in frame_list:
        frame_data_keys = list(json_file[frame].keys())
        for key in frame_data_keys:
            frame_data = json_file[frame][key]
            
            if frame_data.get('text'):
                csv_data.append([frame, key, frame_data['text']['script'], frame_data['emotion']['text']['emotion']])


    with open(OUT_PATH + csv_name, 'w') as csvfile:
        writer = csv.writer(csvfile, delimiter=str(','), lineterminator='\n')
        writer.writerow(['frame', 'frame_key', 'script', 'emotion'])

        for i, row in enumerate(csv_data):
            try:
                writer.writerow(row)
            except:
                print("Exception at row {}!".format(i))


  0%|          | 0/3199 [00:00<?, ?it/s]

### csv 중복 제거 및 통합 

In [6]:
IN_PATH = '/content/drive/MyDrive/CUAI_winter/frame_csv/'
OUT_PATH = '/content/drive/MyDrive/CUAI_winter/'

In [8]:
csv_list = sorted(glob.glob(IN_PATH+'*.csv'))

In [16]:
total_data = pd.DataFrame(columns=['script', 'emotion'])

for csv_name in tqdm(csv_list): # csv 순회
    data = pd.read_csv(csv_name)
    data.drop_duplicates(['script'], keep='first', inplace = True, ignore_index = True)

    data = data[['script', 'emotion']]
    total_data = pd.concat([total_data, data], ignore_index = True)

  0%|          | 0/3199 [00:00<?, ?it/s]

In [17]:
total_data.head()

,script,emotion
0,"야, 여기 있던 내 옷 못 봤어?",neutral
1,무슨 옷 말하는 건데?,fear
2,"그거, 내가 생일에 친구한테 선물받은 원피스!",angry
3,친구한테… 선물 받은 거?,fear
4,"그래, 그 남색에, 어깨 부분에 장식 달린 거 있잖아.",angry


In [18]:
total_data.shape

(45447, 2)

In [19]:
total_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45447 entries, 0 to 45446
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   script   45447 non-null  object
 1   emotion  45447 non-null  object
dtypes: object(2)
memory usage: 710.2+ KB


In [20]:
total_data.to_feather(OUT_PATH + 'clip.ftr')